In [1]:
import tensorflow as tf

2024-11-09 19:04:47.265280: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 19:04:47.347405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 19:04:47.386203: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 19:04:47.397439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 19:04:47.456223: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable()  # Register the class
class MicroF2Score(tf.keras.metrics.Metric):
    def _init_(self, name='micro_f2_score', beta=2, **kwargs):
        super(MicroF2Score, self)._init_(name=name, **kwargs)
        self.beta = beta
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Threshold y_pred to get binary predictions
        y_pred = tf.cast(y_pred > 0.5, tf.float32)
        
        # Cast y_true to float32 to ensure compatibility
        y_true = tf.cast(y_true, tf.float32)

        # Calculate true positives, false positives, and false negatives
        true_positive = tf.reduce_sum(y_true * y_pred)
        false_positive = tf.reduce_sum(y_pred * (1 - y_true))
        false_negative = tf.reduce_sum((1 - y_pred) * y_true)

        # Update the corresponding weights
        self.tp.assign_add(true_positive)
        self.fp.assign_add(false_positive)
        self.fn.assign_add(false_negative)

    def result(self):
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        f_beta = (1 + self.beta*2) * (precision * recall) / (self.beta*2 * precision + recall + tf.keras.backend.epsilon())
        return f_beta

    def reset_states(self):
        self.tp.assign(0)
        self.fp.assign(0)
        self.fn.assign(0)

In [ ]:
import tensorflow as tf
import numpy as np

with tf.device('/device:CPU:0'):

    model = tf.keras.models.load_model('model.keras')

    X = np.load('embeddings_1.npy')
    X_2 = np.load('embeddings_2.npy')
    X = np.concatenate((X, X_2), axis=0)
    # Step 1: Read label data from files (assuming you have already defined this part)
    label_data = []
    file_names = ['icd_codes_1.txt', 'icd_codes_2.txt']  # Update with actual filenames
    for file_name in file_names:
        with open(file_name, 'r') as file:
            label_data.extend(line.strip() for line in file if line.strip())

    # Step 2: Create a set of unique ICD-10 codes for efficient lookup
    unique_codes = set()
    for labels in label_data:
        unique_codes.update(labels.split(";"))
    unique_codes = sorted(unique_codes)  # Convert to a sorted list at the end

    # Step 3: Initialize the StringLookup layer
    lookup_layer = tf.keras.layers.StringLookup(vocabulary=unique_codes, output_mode="multi_hot", mask_token=None,num_oov_indices=0)

    # Step 4: Create a tf.data.Dataset to handle large data efficiently
    label_data_ds = tf.data.Dataset.from_tensor_slices(label_data)
    # Step 5: Define a function to encode each label set
    def encode_labels(labels):
        multi_hot = lookup_layer(tf.strings.split(labels, sep=";"))
        return tf.cast(multi_hot, dtype=tf.int16)  # Reducing the precision here
    # Step 6: Map encoding function over the dataset and batch it
    # Batch processing reduces memory usage
    multi_hot_labels_ds = label_data_ds.map(encode_labels, num_parallel_calls=tf.data.AUTOTUNE).batch(1000)

    # Step 7: Concatenate all batches to get the final `y` tensor
    y = tf.concat(list(multi_hot_labels_ds), axis=0)

    # Ensure the correct shape of `y`
    print("Shape of y:", y.shape)  # Should output: (200000, 1400)

    y = y.numpy()


I0000 00:00:1731159289.358037   48436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731159289.434475   48436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731159289.437525   48436 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731159289.442754   48436 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Shape of y: (198982, 1400)


2024-11-09 19:04:53.473085: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# from sklearn.model_selection import train_test_split

# with tf.device('/device:CPU:0'):
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# from sklearn.metrics import fbeta_score

# # Function to find optimal threshold for each label
# def tune_thresholds(y_true, y_pred, beta=2.0, num_thresholds=50):
#     best_thresholds = []
#     for i in range(y_true.shape[1]):
#         best_f2 = 0
#         best_threshold = 0.5
#         for threshold in np.linspace(0.1, 0.9, num_thresholds):
#             y_pred_bin = (y_pred[:, i] >= threshold).astype(int)
#             f2 = fbeta_score(y_true[:, i], y_pred_bin, beta=beta, average='micro', zero_division=1)
#             if f2 > best_f2:
#                 best_f2 = f2
#                 best_threshold = threshold
#         best_thresholds.append(best_threshold)
#     return best_thresholds

# # Inference function with optimized thresholds for test data
# def predict_with_thresholds(model, X, thresholds):
#     y_pred = model.predict(X)
#     y_pred_bin = np.array([(y_pred[:, i] >= thresholds[i]).astype(int) for i in range(y_pred.shape[1])]).T
#     return y_pred_bin

# with tf.device('/device:CPU:0'):
#     # Get predictions on the validation set
#     y_val_pred = model.predict(X_val)

#     # Find optimal thresholds for each label
#     best_thresholds = tune_thresholds(y_val, y_val_pred)

#     # Apply optimized thresholds to validation predictions
#     y_val_pred_bin = np.array([(y_val_pred[:, i] >= best_thresholds[i]).astype(int) for i in range(y_val_pred.shape[1])]).T

#     # Calculate and print the validation F2 score with optimized thresholds
#     validation_f2 = fbeta_score(y_val, y_val_pred_bin, beta=2, average='micro', zero_division=1)
#     print("Optimized Micro-F2 Score on validation set:", validation_f2)



In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd

with tf.device('/device:CPU:0'):
    # Load best model
    # model = tf.keras.models.load_model('best_model.keras')
    # Load test data
    X_test = np.load('test_data.npy')

    # Predict probabilities and convert to binary using the tuned thresholds
    # y_test_pred = predict_with_thresholds(model, X_test, best_thresholds)
    y_test_pred = model.predict(X_test)

    # Prepare reverse lookup
    lookup_layer = tf.keras.layers.StringLookup(
        vocabulary=unique_codes, invert=True, output_mode="int", mask_token=None, num_oov_indices=0
    )

    # Convert predictions to ICD10 codes
    predicted_indices = [np.where(pred_row == 1)[0] for pred_row in y_test_pred]
    predicted_codes = [lookup_layer(indices).numpy() for indices in predicted_indices]
    predicted_codes = [[code.decode('utf-8') for code in row] for row in predicted_codes]
    predicted_labels = [';'.join(row) for row in predicted_codes]

    # Create and save submission DataFrame
    submission_df = pd.DataFrame({
        'id': range(1, len(predicted_labels) + 1),
        'labels': predicted_labels
    })
    submission_df.to_csv('submission_srini.csv', index=False)

  28/3110 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step     

I0000 00:00:1731159294.068067   48604 service.cc:146] XLA service 0x7fe99800a220 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731159294.068098   48604 service.cc:154]   StreamExecutor device (0): Host, Default Version
2024-11-09 19:04:54.081015: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731159294.172145   48604 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3110/3110 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
